<a href="https://colab.research.google.com/github/Loop3D/2022_iamg_workshop/blob/main/Utility_1_Config_file_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## hjson Config file generator
This simple notebook provides a tool for generating map2loop config hjson files and a minimum python script, one of the more annoying parts of the process.
   
1. It first gets the project name and destination projection system (as EPSG number)   
   
2. It then asks for the paths to the different shape files (for WFS you will have to edit this manually afterwards)   
   
3. Next for each input file, you have to define which fields to use, and which keywords to look for in some of these fields 

4. Finally you can alter the user definable parameters
     
5. This generates a minimum python script for your data with all the paths and parameters hopefully correctly configured.This can be run as is or pasted into a new notebook. 


In [ ]:
! git clone https://github.com/Loop3D/map2loop2-notebooks
!pip install ipyfilechooser
!pip install git+https://github.com/geopandas/geopandas.git@v0.10.2


In [ ]:
#if not already installed, uncomment next line, run, (then recomment line)
#!pip install ipyfilechooser 
from ipyfilechooser import FileChooser
import geopandas as gpd
import ipywidgets as widgets
import pandas as pd
import os
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))
    
pd.options.display.max_columns = None
style = {'description_width': 'initial'}
c_l= {
#Orientations
  "d": "dip",                  #field that contains dip information
  "dd": "azimuth",             #field that contains dip direction information
  "sf": 'codedescpt',             #field that contains information on type of structure
  "bedding": 'bedding',            #text to search for in field defined by sf code to show that this is a bedding measurement
  "otype": 'dip direction',            #flag to determine measurement convention (currently 'strike' or 'dip direction')
  "bo": "deform",             #field that contains type of foliation
  "btype": 'BEOI',            #text to search for in field defined by bo code to show that this is an overturned bedding measurement
#stratigraphy
  "g": 'sub_provin',               #field that contains coarser stratigraphic coding
  "g2": 'province',              #field that contains alternate coarser stratigraphic coding if 'g' is blank
  "c": 'unit_name',                 #field that contains finer stratigraphic coding
  "ds": 'deposition',           #field that contains information about lithology
  "u": 'nsw_code',             #field that contains alternate stratigraphic coding (not used??)
  "r1": 'class',           #field that contains  extra lithology information
  "r2": 'igneous_ty',           #field that contains even more lithology information
  "sill": 'sill',              #text to search for in field defined by ds code to show that this is a sill
  "intrusive": 'intrusive',    #text to search for in field defined by r1 code to show that this is an intrusion
  "volcanic": 'volcanic',      #text to search for in field defined by ds code to show that this is an volcanic (not intrusion)
#mineral deposits
  "msc": 'SITE_CODE',          #field that contains site code of deposit
  "msn": 'SHORT_NAME',         #field that contains short name of deposit
  "mst": 'SITE_TYPE_',         #field that contains site type of deposit
  "mtc": 'TARGET_COM',         #field that contains target commodity of deposit
  "mscm": 'SITE_COMMO',        #field that contains site commodity of deposit
  "mcom": 'COMMODITY_',        #field that contains commodity group of deposit
  "minf": 'Infrastructure',    #text to search for in field defined by mst code that shows site to ignore
#timing
  "min": 'top_end_ag',         #field that contains minimum age of unit defined by ccode
  "max": 'base_start',         #field that contains maximum age of unit defined by ccode
#faults and folds
  "f": 'boundaryty',              #field that contains information on type of structure
  "fault": 'Fault',            #text to search for in field defined by f code to show that this is a fault
  "ff": 'codedescpt',  # field that contains information on type of structure
  "fold": 'cline',  #text to search for in field defined by ff code to show that this is a fold axial trace
  "fdip": 'dip',               # field for numeric fault dip value
  "fdipnull": '0',         # text to search for in field defined by fdip to show that this has no known dip
  "fdipdir": 'faultdipdi',        # field for text fault dip direction value 
  "fdipdir_flag": 'alpha',        # flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast    
  "fdipest": 'faultdipan',        # field for text fault dip estimate value
  "fdipest_vals": 'Moderate,Listric,Steep,Vertical',        # text to search for in field defined by fdipest to give fault dip estimate in increasing steepness
  "n": 'name',                 #field that contains information on name of fault (not used??)
  "t": 'codedescpt',                 #field that contains information on type of fold
  "syn": 'Syncline',           #text to search for in field defined by t to show that this is a syncline
#ids
  "o": 'unique_id',             #field that contains unique id of geometry object
  "gi": 'objectid',            #field that contains unique id of structure point
  "deposit_dist":500           # dist between mindep and contact to be considered "on contact"
}

In [ ]:
test_data_name_choice=widgets.Text(
    placeholder='Test',
    value='Test',
    description='Project Name:',
    disabled=False,
    style=style,
    tooltip='Name of directory to hold config file (relative path)'
)
display(test_data_name_choice)

epsg=pd.read_csv('/content/map2loop2-notebooks/source_data/epsg.csv')

crs_choice=widgets.Dropdown(
    options=epsg['epsg'],
    description='CRS EPSG:',
    disabled=False
)
display(crs_choice)


In [ ]:
test_data_name=test_data_name_choice.value
epsg_code=crs_choice.value.split(":")
dst_crs='epsg:'+epsg_code[0]
printmd('**'+test_data_name+", "+dst_crs+'**')

In [ ]:
minx_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min x:',
    disabled=False,
    style=style
)
display(minx_choice)

maxx_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max x:',
    disabled=False,
    style=style
)
display(maxx_choice)

miny_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min y:',
    disabled=False,
    style=style
)
display(miny_choice)

maxy_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max y:',
    disabled=False,
    style=style
)
display(maxy_choice)

minz_choice=widgets.Text(
    placeholder='0',
    value='0',
    description='min z:',
    disabled=False,
    style=style
)
display(minz_choice)

maxz_choice=widgets.Text(
    placeholder='1',
    value='1',
    description='max z:',
    disabled=False,
    style=style
)
display(maxz_choice)


In [ ]:
minx=minx_choice.value
maxx=maxx_choice.value
miny=miny_choice.value
maxy=maxy_choice.value
minz=minz_choice.value
maxz=maxz_choice.value


structure_file_m = FileChooser('../')
structure_file_m.title = '<b>Choose Bedding Orientation point file</b>'
display(structure_file_m)

fault_file_m = FileChooser('../')
fault_file_m.title = '<b>Choose Fault polyline file</b>'
display(fault_file_m)

fold_file_m = FileChooser('../')
fold_file_m.title = '<b>Choose Fold polyline file</b>'
display(fold_file_m)

geology_file_m = FileChooser('../')
geology_file_m.title = '<b>Choose Geology polygon file</b>'
display(geology_file_m)

mindep_file_m = FileChooser('../')
mindep_file_m.title = '<b>Choose Mineral Deposit point file (or use null_mindeps.shp from source_data directory)</b>'
mindep_file_m.default_filename = 'null_mindeps.shp'
display(mindep_file_m)

dtm_file_m = FileChooser('../')
dtm_file_m.title = '<b>Choose dtm geotif</b>'
display(dtm_file_m)
display('or leave blank if in Australia')


In [ ]:
if(str(structure_file_m.selected_path) =='None'):
    structure_file=''
else:
    structure_file=structure_file_m.selected_path.replace('\\','/')+'/'+structure_file_m.selected_filename
if(str(fault_file_m.selected_path) =='None'):
    fault_file=''
else:
    fault_file=fault_file_m.selected_path.replace('\\','/')+'/'+fault_file_m.selected_filename
if(str(fold_file_m.selected_path) =='None'):
    fold_file=''
else:
    fold_file=fold_file_m.selected_path.replace('\\','/')+'/'+fold_file_m.selected_filename
if(str(geology_file_m.selected_path) =='None'):
    geology_file=''
else:
    geology_file=geology_file_m.selected_path.replace('\\','/')+'/'+geology_file_m.selected_filename
if(str(mindep_file_m.selected_path) =='None'):
    mindep_file=''
else:
    mindep_file=mindep_file_m.selected_path.replace('\\','/')+'/'+mindep_file_m.selected_filename
if(str(dtm_file_m.selected_path) =='None'):
    dtm_file='http://services.ga.gov.au/gis/services/DEM_SRTM_1Second_over_Bathymetry_Topography/MapServer/WCSServer?'
else:
    dtm_file=dtm_file_m.selected_path.replace('\\','/')+'/'+dtm_file_m.selected_filename

display(structure_file,fault_file,fold_file,geology_file,mindep_file,dtm_file)

In [ ]:
if(not structure_file==''):
    structure=gpd.read_file(structure_file)
    display(structure.head())
    
    cols=['No_col']
    for sc in structure.columns:
        cols.append(sc)
        
    printmd('**Based on file: '+structure_file+'**')
    printmd('\n**Field that contains unique id of structure point**')
    gi_choice=widgets.Dropdown(
        options=cols,
        description='Unique ID',
        disabled=False

    )
    display(gi_choice)

    printmd('**Field that contains information on type of structure (e.g. bedding, cleavage etc.)**')

    sf_choice=widgets.Dropdown(
        options=cols,
        description='Structure',
        disabled=False
    )
    display(sf_choice)

    printmd('**Text to search for in field defined by type of structure code to show that this is a bedding measurement**')

    bedding_choice=widgets.Text(
        placeholder='Bed',
        value='Bed',
        description='Bedding text',
        disabled=False
    )
    display(bedding_choice)

    printmd('**Field that contains dip information**')

    d_choice=widgets.Dropdown(
        options=cols,
        description='Dip',
        disabled=False
    )
    display(d_choice)

    printmd('**Field that contains dip direction or strike information**')

    dd_choice=widgets.Dropdown(
        options=cols,
        description='Dip Direction',
        disabled=False
    )
    display(dd_choice)

    printmd('**Flag to determine measurement convention**')

    otype_choice=widgets.Dropdown(
        options=('dip direction','strike'),
        description='Convention',
        disabled=False
    )
    display(otype_choice)

    printmd('**Field that contains polarity of foliation (upward facing or not)**')

    bo_choice=widgets.Dropdown(
        options=cols,
        description='Polarity',
        disabled=False
    )
    display(bo_choice)

    printmd('**Text to search for in field defined by Polarity code to show that this is an overturned bedding measurement**')

    btype_choice=widgets.Text(
        placeholder='Blah',
        value='Blah',
        description='Pol. text',
        disabled=False
    )
    display(btype_choice)



In [ ]:
c_l['gi']=gi_choice.value
c_l['sf']=sf_choice.value
c_l['bedding']=bedding_choice.value
c_l['d']=d_choice.value
c_l['dd']=dd_choice.value
c_l['otype']=otype_choice.value
c_l['bo']=bo_choice.value
c_l['btype']=btype_choice.value

if(not fault_file==''):
    fault=gpd.read_file(fault_file)
    display(fault.head())
    printmd('**Based on file: '+fault_file+'**')

    cols=['No_col']
    for sc in fault.columns:
        cols.append(sc)
        
    printmd('\n**Field that contains information on type of structure (fault, fold axial trace etc.)**')


    f_choice=widgets.Dropdown(
        options=cols,
        description='Fault field:',
        disabled=False
    )
    display(f_choice)

    printmd('**Text to search for in field defined by Fault field code to show that this is a fault**')

    fault_choice=widgets.Text(
        placeholder='Fault',
        value='Fault',
        description='Fault text:',
        disabled=False
    )
    display(fault_choice)

    printmd('**Field for numeric fault dip value**')

    fdip_choice=widgets.Dropdown(
        options=cols,
        description='Dip:',
        disabled=False
    )
    display(fdip_choice)

    printmd('**Text to search for in field defined by Dip to show that this has no known dip**')

    fdipnull_choice=widgets.Text(
        placeholder='0',
        value='0',
        description='NULL dip:',
        disabled=False
    )
    display(fdipnull_choice)

    printmd('**Field for text fault dip direction value (southeast etc.)**')

    fdipdir_choice=widgets.Dropdown(
        options=cols,
        description='Dip Direction:',
        disabled=False
    )
    display(fdipdir_choice)

    printmd('**Flag for text fault dip direction type num e.g. 045 or alpha e.g. southeast**')

    fdipdir_flag_choice=widgets.Dropdown(
        options=('alpha','num'),
        description='Dip Convention:',
        disabled=False,
        style=style
    )
    display(fdipdir_flag_choice)

    printmd('**Field for text fault dip estimate value (Moderate, Steep etc.)**')

    fdipest_choice=widgets.Dropdown(
        options=cols,
        description='Dip estimate:',
        disabled=False
    )
    display(fdipest_choice)

In [ ]:
printmd('**Text to search for in field defined by fdipest to give fault dip estimate in increasing steepness**')

if(fdipest_choice.value=='No_col'):
    c_l['fdipest_vals']='No_col'
else:
    fdipest_vals_choice=widgets.Text(
        placeholder='Sorted dip order (shallow to steep)',
        value=str(fault[fdipest_choice.value].unique()).replace("[","").replace("]","").replace(" ",","),
        description='Text Dips:',
        disabled=False
    )
    #display(fdipest_vals)
    c_l['fdipest_vals']=fdipest_vals_choice.value


    
printmd('**Field that contains information on name of fault (not used??)**')

n_choice=widgets.Dropdown(
    options=cols,
    description='Name:',
    disabled=False
)
display(n_choice)

In [ ]:
c_l['f']=f_choice.value
c_l['fault']=fault_choice.value
c_l['fdip']=fdip_choice.value
c_l['fdipnull']=fdipnull_choice.value
c_l['fdipdir']=fdipdir_choice.value
c_l['fdipdir_flag']=fdipdir_flag_choice.value
c_l['fdipest']=fdipest_choice.value
c_l['n']=n_choice.value

if(not fold_file==''):
    fold=gpd.read_file(fold_file)
    display(fold.head())
    cols=['No_col']
    for sc in fold.columns:
        cols.append(sc)
        
    printmd('**Based on file:'+' '+fold_file+'**')

    printmd('\n**Field that contains information on type of structure (fold, fault etc.)**')

    ff_choice=widgets.Dropdown(
        options=cols,
        description='Fold field:',
        disabled=False
    )
    display(ff_choice)

    printmd('**Text to search for in field defined by Fold field to show that this is a fold axial trace**')

    fold_choice=widgets.Text(
        placeholder='Fold',
        value='Fold',
        description='Fold text:',
        disabled=False
    )
    display(fold_choice)

    printmd('**Field that contains information on type of fold (anticline, syncline)**')

    t_choice=widgets.Dropdown(
        options=cols,
        description='Polarity field:',
        disabled=False
    )
    display(t_choice)

    printmd('**Text to search for in field defined by Polarity Field to show that this is a syncline**')

    syn_choice=widgets.Text(
        placeholder='syn',
        value='syn',
        description='Syncline text:',
        disabled=False
    )
    display(syn_choice)


In [ ]:
c_l['ff']=ff_choice.value
c_l['fold']=fold_choice.value
c_l['t']=t_choice.value
c_l['syn']=syn_choice.value

if(not geology_file==''):
    geology=gpd.read_file(geology_file)
    display(geology.head())
    cols=['No_col']
    for sc in geology.columns:
        cols.append(sc)
    printmd('**Based on file:'+' '+geology_file+'**')

    printmd('\n**Field that contains unique ID**')

    o_choice=widgets.Dropdown(
        options=cols,
        description='Unique ID',
        disabled=False
    )
    display(o_choice)

    printmd('**Field that contains finer stratigraphic coding**')

    c_choice=widgets.Dropdown(
        options=cols,
        description='Formation',
        disabled=False
    )
    display(c_choice)

    printmd('\n**Field that contains coarser stratigraphic coding**')

    g_choice=widgets.Dropdown(
        options=cols,
        description='Group',
        disabled=False
    )
    display(g_choice)

    printmd('**Field that contains alternate coarser stratigraphic coding if Group is blank**')

    g2_choice=widgets.Dropdown(
        options=cols,
        description='Group alt',
        disabled=False
    )
    display(g2_choice)


    printmd('**Field that contains alternate stratigraphic coding (used to check for "drift" formations'')**')

    u_choice=widgets.Dropdown(
        options=cols,
        description='Strat alt',
        disabled=False
    )
    display(u_choice)

    printmd('**Field that contains information about lithology**')

    ds_choice=widgets.Dropdown(
        options=cols,
        description='Description:',
        disabled=False
    )
    display(ds_choice)

    printmd('**Field that contains extra lithology information**')

    r1_choice=widgets.Dropdown(
        options=cols,
        description='Litho',
        disabled=False
    )
    display(r1_choice)

    printmd('**Field that contains even more lithology information**')

    r2_choice=widgets.Dropdown(
        options=cols,
        description='Litho alt',
        disabled=False
    )
    display(r2_choice)

    printmd('**Text to search for in field defined by Description code to show that this is a sill**')

    sill_choice=widgets.Text(
        placeholder='sill',
        value='sill',
        description='Sill text',
        disabled=False
    )
    display(sill_choice)

    printmd('**Text to search for in field defined by Litho code to show that this is an intrusion**')

    intrusive_choice=widgets.Text(
        placeholder='intrusive',
        value='intrusive',
        description='Intrusive text',
        disabled=False
    )
    display(intrusive_choice)

    printmd('**Text to search for in field defined by Description code to show that this is an volcanic (not intrusion)**')


    volcanic_choice=widgets.Text(
        placeholder='volc',
        value='volc',
        description='Volc text',
        disabled=False
    )
    display(volcanic_choice)

    printmd('**Field that contains minimum age of unit defined by Formation**')

    min_choice=widgets.Dropdown(
        options=cols,
        description='Min age',
        disabled=False
    )
    display(min_choice)

    printmd('**Field that contains maximum age of unit defined by Formation**')

    max_choice=widgets.Dropdown(
        options=cols,
        description='Max age',
        disabled=False
    )
    display(max_choice)



In [ ]:
c_l['o']=o_choice.value
c_l['g']=g_choice.value
c_l['g2']=g2_choice.value
c_l['c']=c_choice.value
c_l['ds']=ds_choice.value
c_l['u']=u_choice.value
c_l['r1']=r1_choice.value
c_l['r2']=r2_choice.value
c_l['sill']=sill_choice.value
c_l['intrusive']=intrusive_choice.value
c_l['volcanic']=volcanic_choice.value
c_l['min']=min_choice.value
c_l['max']=max_choice.value

mindep=gpd.read_file(mindep_file)
display(mindep.head())
cols=['No_col']
for sc in mindep.columns:
    cols.append(sc)

printmd('**Based on file:'+' '+mindep_file+'**')

printmd('\n**Field that contains site code of deposit**')

msc_choice=widgets.Dropdown(
    options=cols,
    value='SITE_CODE',
    description='Site code',
)
display(msc_choice)

printmd('**Field that contains short name of deposit (e.g. Jones deposit)**')

msn_choice=widgets.Dropdown(
    options=cols,
    value='SHORT_NAME',
    description='Short Name',
)
display(msn_choice)

printmd('**Field that contains site type of deposit (Mine, occurrence etc.)**')

mst_choice=widgets.Dropdown(
    options=cols,
    value='SITE_TYPE_',
    description='Site type',
)
display(mst_choice)


printmd('**Field that contains target commodity of deposit (Commodity)**')

mtc_choice=widgets.Dropdown(
    options=cols,
    value='TARGET_COM',
    description='Target Comm',
)
display(mtc_choice)

printmd('**Field that contains site commodity of deposit (ore Mineral)**')

mscm_choice=widgets.Dropdown(
    options=cols,
    value='SITE_COMMO',
    description='Site Commodity',
)
display(mscm_choice)

printmd('**Field that contains commodity group of deposit (e.g. Metal)**')

mcom_choice=widgets.Dropdown(
    options=cols,
    value='COMMODITY_',
    description='Commodity',
    style=style
)
display(mcom_choice)

printmd('**Text to search for in field defined by Site type code that shows site to ignore**')

minf_choice=widgets.Text(
    placeholder='Cons',
    value='CONS',
    description='Not mine text',
)
display(minf_choice)


In [ ]:
c_l['msc']=msc_choice.value
c_l['msn']=msn_choice.value
c_l['mst']=mst_choice.value
c_l['mtc']=mtc_choice.value
c_l['mscm']=mscm_choice.value
c_l['mcom']=mcom_choice.value
c_l['minf']=minf_choice.value

if(not os.path.isdir(test_data_name)):
    os.mkdir(test_data_name)
f=open(test_data_name+'/'+test_data_name+'_config.hjson',"w")
f.write(str(c_l).replace(',',',\n'))
f.write("\n")

f.close()

In [ ]:
c_l

In [ ]:
printmd('**aus: Indicates if area is in Australia for using ASUD. Defaults to True.**')

aus_choice=widgets.Text(
    value='True',
    description='aus',
    disabled=False
)
display(aus_choice)

printmd('**deposits: Mineral deposit names for focused topology extraction. Defaults to "Fe,Cu,Au,NONE".**')

deposits_choice=widgets.Text(
    value='"Fe,Cu,Au,NONE"',
    description='deposits',
    disabled=False
)
display(deposits_choice)

printmd('**dtb: Path to depth to basement grid. Defaults to ''.**')

dtb_choice=widgets.Text(
    value='',
    description='dtb',
    disabled=False
)
display(dtb_choice)

printmd('**orientation_decimate: Save every nth orientation data point. Defaults to 0.**')

orientation_decimate_choice=widgets.Text(
    value='0',
    description='orientation_decimate',
    disabled=False
)
display(orientation_decimate_choice)

printmd('**contact_decimate: Save every nth contact data point. Defaults to 5.**')

contact_decimate_choice=widgets.Text(
    value='5',
    description='contact_decimate',
    disabled=False
)
display(contact_decimate_choice)

printmd('**intrusion_mode: 1 to exclude all intrusions from basal contacts, 0 to only exclude sills. Defaults to 0.**')

intrusion_mode_choice=widgets.Text(
    value='0',
    description='intrusion_mode',
    disabled=False
)
display(intrusion_mode_choice)

printmd('**interpolation_spacing: Interpolation grid spacing in meters. Defaults to 500.**')
interpolation_spacing_choice=widgets.Text(
    value= '500',
    description='interpolation_spacing',
    disabled=False
)
display(interpolation_spacing_choice)

printmd('**misorientation: maximimum misorientation between poles to bedding girdles to allow to be part of same supergroup. Defaults to 30.**')

misorientation_choice=widgets.Text(
    value='30',
    description='misorientation',
    disabled=False
)
display(misorientation_choice)

printmd('**interpolation_scheme: What interpolation function to use of scipy_rbf (radial basis) or scipy_idw (inverse distance weighted). Defaults to \'scipy_rbf\'.**')

interpolation_scheme_choice=widgets.Text(
    value='scipy_rbf',
    description='interpolation_scheme',
    disabled=False
)
display(interpolation_scheme_choice)

printmd('**fault_decimate: Save every nth fault data point. Defaults to 5.**')

fault_decimate_choice=widgets.Text(
    value='5',
    description='fault_decimate',
    disabled=False
)
display(fault_decimate_choice)

printmd('**min_fault_length: Min fault length to be considered. Defaults to 5000.**')

min_fault_length_choice=widgets.Text(
    value='5000',
    description='min_fault_length',
    disabled=False
)
display(min_fault_length_choice)

printmd('**fault_dip: Defaults to 90.**')

fault_dip_choice=widgets.Text(
    value='90',
    description='fault_dip',
    disabled=False
)
display(fault_dip_choice)

printmd('**pluton_dip: Dip of contacts. Defaults to 45.**')

pluton_dip_choice=widgets.Text(
    value='45',
    description='pluton_dip',
    disabled=False
)
display(pluton_dip_choice)
options=['domes','saucers','batholiths','pendants']

printmd('**pluton_form: Possible forms from domes, saucers or pendant. Defaults to \'domes\'.**')
pluton_form_choice=widgets.Dropdown(
    options=options,
    value='domes',
    description='pluton_form:',
    disabled=False,
)
display(pluton_form_choice)

printmd('**dist_buffer: Buffer for processing plutons for faults that do not quite touch pluton. Defaults to 10.**')

dist_buffer_choice=widgets.Text(
    value='10',
    description='dist_buffer',
    disabled=False
)
display(dist_buffer_choice)

printmd('**contact_dip: Assign fixed dip to all contacts, -99 means interpolated local orientaiton field is used. Defaults to -999.**')

contact_dip_choice=widgets.Text(
    value='-999',
    description='contact_dip',
    disabled=False
)
display(contact_dip_choice)

printmd('**contact_orientation_decimate: Save every nth contact orientation point. Defaults to 5.**')

contact_orientation_decimate_choice=widgets.Text(
    value='5',
    description='contact_orientation_decimate',
    disabled=False
)
display(contact_orientation_decimate_choice)

printmd('**null_scheme: How null values present in the dtb. Defaults to \'null\'.**')

null_scheme_choice=widgets.Text(
    value='null',
    description='null_scheme',
    disabled=False
)
display(null_scheme_choice)

printmd('**thickness_buffer: Maximum distance to search for next contact when calculating formation thickness. Defaults to 5000.**')

thickness_buffer_choice=widgets.Text(
    value='5000',
    description='thickness_buffer',
    disabled=False
)
display(thickness_buffer_choice)

printmd('**max_thickness_allowed: Maximum formation thickness allowed for calculation. Defaults to 10000.**')

max_thickness_allowed_choice=widgets.Text(
    value='10000',
    description='max_thickness_allowed',
    disabled=False
)
display(max_thickness_allowed_choice)

printmd('**fold_decimate: Save every nth fold data point. Defaults to 5.**')

fold_decimate_choice=widgets.Text(
    value='5',
    description='fold_decimate',
    disabled=False
)
display(fold_decimate_choice)

printmd('**fat_step: How much to step out normal to the fold axial trace for limb orientations. Defaults to 750.**')

fat_step_choice=widgets.Text(
    value='750',
    description='fat_step',
    disabled=False
)
display(fat_step_choice)

printmd('**close_dip: Dip to assign to all new orientations. Defaults to -999.**')

close_dip_choice=widgets.Text(
    value='-999',
    description='close_dip',
    disabled=False
)
display(close_dip_choice)

printmd('**use_interpolations: Defaults to False.**')

use_interpolations_choice=widgets.Text(
    value='False',
    description='use_interpolations',
    disabled=False
)
display(use_interpolations_choice)

printmd('**use_fat: Use fake limb orientation data around fold hinges Defaults to False.**')

use_fat_choice=widgets.Text(
    value='False',
    description='use_fat',
    disabled=False
)
display(use_fat_choice)

printmd('**clut_path: User defined look up table for formation colours Default \'\'.**')

clut_path_choice=widgets.Text(
    value='',
    description='clut_path',
    disabled=False
)
display(clut_path_choice)

printmd('**fault_orientation_clusters: Number of kmeans fault orientation clusters Defaults to 2.**')

fault_orientation_clusters_choice=widgets.Text(
    value='False',
    description='# orient clusters',
    disabled=False
)
display(fault_orientation_clusters_choice)

printmd('**fault_length_clusters: Number of kmeans fault length clusters Defaults to 2.**')

fault_length_clusters_choice=widgets.Text(
    value='False',
    description='# length clusters',
    disabled=False
)
display(fault_length_clusters_choice)

printmd('**use_roi_clip: Use arbitrary polygon to clip data Default False.**')

use_roi_clip_choice=widgets.Text(
    value='False',
    description='Use polygon for ROI',
    disabled=False
)
display(use_roi_clip_choice)

printmd('**roi_clip_path: Path to clipping ROI Default \'\'.**')

roi_clip_path_choice=widgets.Text(
    value='',
    description='ROI_path',
    disabled=False
)
display(roi_clip_path_choice)





In [ ]:
aus=aus_choice.value
deposits=deposits_choice.value
dtb=dtb_choice.value
orientation_decimate=orientation_decimate_choice.value
contact_decimate=contact_decimate_choice.value
intrusion_mode=intrusion_mode_choice.value
interpolation_spacing=interpolation_spacing_choice.value
misorientation=misorientation_choice.value
interpolation_scheme=interpolation_scheme_choice.value
fault_decimate=fault_decimate_choice.value
min_fault_length=min_fault_length_choice.value
fault_dip=fault_dip_choice.value
pluton_dip=pluton_dip_choice.value
pluton_form=pluton_form_choice.value
dist_buffer=dist_buffer_choice.value
contact_dip=contact_dip_choice.value
contact_orientation_decimate=contact_orientation_decimate_choice.value
null_scheme=null_scheme_choice.value
thickness_buffer=thickness_buffer_choice.value
max_thickness_allowed=max_thickness_allowed_choice.value
fold_decimate=fold_decimate_choice.value
fat_step=fat_step_choice.value
close_dip=close_dip_choice.value
use_interpolations=use_interpolations_choice.value
use_fat=use_fat_choice.value
clut_path=clut_path_choice.value
fault_orientation_clusters=fault_orientation_clusters_choice.value
fault_length_clusters=fault_length_clusters_choice.value
use_roi_clip=use_roi_clip_choice.value
roi_clip_path=roi_clip_path_choice.value

                       


In [ ]:
f=open(test_data_name+'/run_m2l.py',"w")
f.write("\
\
import os\n\
from map2loop.project import Project\n\
\n\
proj = Project(\n\
                  geology_filename='"+geology_file+"',\n\
                  fault_filename='"+fault_file+"',\n\
                  fold_filename='"+fold_file+"',\n\
                  structure_filename='"+structure_file+"',\n\
                  mindep_filename='"+mindep_file+"',\n\
                  dtm_filename='"+dtm_file+"',\n\
                  metadata_filename='./"+test_data_name+"'_config.hjson'\n\
                  overwrite='true',\n\
                  verbose_level=VerboseLevel.NONE,\n\
                  project_path='"+test_data_name+"',\n\
                  working_projection='"+dst_crs+"',\n\
                )\n\
\n\
proj.update_config(\n\
                    out_dir='"+test_data_name+"',\n\
                    bbox_3d={\n\
                        'minx': "+minx+" ,\n\
                        'miny': "+miny+" ,\n\
                        'maxx': "+maxx+" ,\n\
                        'maxy': "+maxy+" ,\n\
                        'base': "+minz+",\n\
                        'top':  "+maxz+"},\n\
                    run_flags={'local' : True,\n\
                        'aus': "+aus+",\n\
                        'close_dip': "+close_dip+",\n\
                        'contact_decimate': "+contact_decimate+",\n\
                        'contact_dip': "+contact_dip+",\n\
                        'contact_orientation_decimate': "+contact_orientation_decimate+",\n\
                        'deposits': "+deposits+",\n\
                        'dist_buffer': "+dist_buffer+",\n\
                        'dtb': '"+dtb+"',\n\
                        'fat_step': "+fat_step+",\n\
                        'fault_decimate': "+fault_decimate+",\n\
                        'fault_dip': "+fault_dip+",\n\
                        'fold_decimate': "+fold_decimate+",\n\
                        'interpolation_scheme': '"+interpolation_scheme+"',\n\
                        'interpolation_spacing': "+interpolation_spacing+",\n\
                        'intrusion_mode': "+intrusion_mode+",\n\
                        'max_thickness_allowed': "+max_thickness_allowed+",\n\
                        'min_fault_length': "+min_fault_length+",\n\
                        'misorientation': "+misorientation+",\n\
                        'null_scheme': '"+null_scheme+"',\n\
                        'orientation_decimate': "+orientation_decimate+",\n\
                        'pluton_dip': "+pluton_dip+",\n\
                        'pluton_form': '"+pluton_form+"',\n\
                        'thickness_buffer': "+thickness_buffer+",\n\
                        'use_fat': "+use_fat+",\n\
                        'use_interpolations': "+use_interpolations+",\n\
                        'fault_orientation_clusters':"+fault_orientation_clusters+",\n\
                        'fault_length_clusters':"+fault_length_clusters+",\n\
                        'use_roi_clip':"+use_roi_clip+",\n\
                        'roi_clip_path':'"+roi_clip_path+"',\n\
                         },\n\
                    proj_crs={'init': '"+dst_crs+"'},\n\
                    clut_path= '"+clut_path+"',\n\
                  )\n\
\nproj.run()\n"   
        
)
f.write("\n")
f.close()